In [30]:
import os
import re
import struct
import io
import tempfile
from wand.image import Image as WandImage

from PIL import Image

ACFA_THUMBNAIL_HEADER = bytes([
    0x10, 0x00, 0x00, 0x80, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x40, 0x00
])



def parse_part_mapping(file_path):
    part_mapping = {}
    current_category = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            # Detect category header like 'Head (0):'
            if line.endswith('):'):
                category_name, category_id = line[:-2].rsplit('(', 1)
                current_category = category_name.strip()
                part_mapping[current_category] = {}
                continue

            if current_category is None:
                continue  # Skip any lines before first category

            # Split the line into part_id and part_name
            if ' ' in line:
                part_id, part_name = line.split(' ', 1)
                part_mapping[current_category][part_id.strip()
                                               ] = part_name.strip()

    return part_mapping


part_mapping = parse_part_mapping("ACFA_PS3_US_PARTID_TO_PARTNAME.txt")
print(part_mapping.keys())

BLOCK_SIZE = 24280
NAME_SIZE = 96  # 48 wchar_t = 96 bytes in UTF-16


def load_file(path):
    with open(path, "rb") as f:
        return f.read()


def save_file(path, data):
    with open(path, "wb") as f:
        f.write(data)

def linear_utf16_clean_name_reader(data, start_offset, max_bytes=96):
    raw_field = data[start_offset:start_offset + max_bytes]
    try:
        decoded = raw_field.decode('utf-16-le', errors='ignore').strip('\x00')
        match = re.match(r'^[A-Za-z0-9 ]+', decoded)
        if match:
            return match.group(0).strip()
        return "<Invalid UTF-16 Encoding>"
    except UnicodeDecodeError:
        return "<Invalid UTF-16 Encoding>"


def read_timestamp(data, offset):
    timestamp_bytes = data[offset:offset + 8]
    return struct.unpack(">Q", timestamp_bytes)[0]


def extract_active_schematic_blocks(file_path):
    """
    Extracts all schematic blocks from the given file.
    Returns a list of blocks.
    """
    data = load_file(file_path)
    schematic_count = data[5]
    blocks = []
    first_marker_offset = 0x148

    for slot_index in range(schematic_count):
        block_start = first_marker_offset + (slot_index * BLOCK_SIZE)
        block = data[block_start:block_start + BLOCK_SIZE]
        blocks.append(block)
    return blocks


def display_schematic_info(block):
    """
    Displays the schematic information from a block.
    Returns a dictionary with the schematic information.
    """
    schematic_name = linear_utf16_clean_name_reader(block, 1, NAME_SIZE)
    designer_name = linear_utf16_clean_name_reader(
        block, 1 + NAME_SIZE, NAME_SIZE)
    timestamp = read_timestamp(block, 192)

    protect_category_byte = block[200]
    protect = (protect_category_byte & 0b10000000) >> 7
    category = (protect_category_byte & 0b01111111) + 1

    parts = extract_parts(block, part_mapping)
    tuning = extract_tuning(block)

    schematic_info = {
        "name": schematic_name,
        "designer": designer_name,
        "category": category,
        "timestamp": timestamp,
        "parts": parts,
        "tuning": tuning
    }

    # Optional print for debugging
    # print(f"  Name: {schematic_info['name']}")
    # print(f"  Designer: {schematic_info['designer']}")
    # print(f"  Protect: {schematic_info['protect']}")
    # print(f"  User Slot: {schematic_info['category']}")
    # print(f"  Raw Byte: 0x{schematic_info['raw_byte']:02x}")
    # print(f"  Timestamp: {schematic_info['timestamp']}")
    # print("  Parts:", schematic_info['parts'])
    # print("  Tuning:", schematic_info['tuning'])
    # print()

    return schematic_info


def extract_parts(block, part_name_lookup):
    LOCAL_PARTS_OFFSET = 0xD8  # 0x220 - 0x148
    PART_ENTRY_SIZE = 2

    # Define lookup keys and display labels separately
    lookup_keys = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Arm Unit', 'Arm Unit', 'Back Unit', 'Back Unit', 'Shoulder Unit'
    ]

    display_labels = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Right Arm Unit', 'Left Arm Unit', 'Right Back Unit',
        'Left Back Unit', 'Shoulder Unit'
    ]

    parts_info = []
    for i, (lookup_key, display_label) in enumerate(zip(lookup_keys, display_labels)):
        offset = LOCAL_PARTS_OFFSET + i * PART_ENTRY_SIZE
        part_id_bytes = block[offset:offset + PART_ENTRY_SIZE]

        if len(part_id_bytes) != 2:
            part_id_str = "<Invalid>"
            part_name = "<Invalid>"
        else:
            part_id_num = int.from_bytes(part_id_bytes, byteorder='big')
            part_id_str = f"{part_id_num:04d}"
            part_name = part_name_lookup.get(lookup_key, {}).get(
                part_id_str, f"Unknown ID {part_id_str}")

        parts_info.append({
            "category": display_label,
            "part_id": part_id_str,
            "part_name": part_name
        })

    return parts_info


def extract_tuning(block):
    LOCAL_TUNING_OFFSET = 0x126  # 0x26E - 0x148
    TUNING_SIZE = 32  # 0x20 bytes

    tuning_labels = [
        'en_output',
        'en_capacity',
        'kp_output',
        'load',
        'en_weapon_skill',
        'maneuverability',
        'firing_stability',
        'aim_precision',
        'lock_speed',
        'missile_lock_speed',
        'radar_refresh_rate',
        'ecm_resistance',
        'rectification_head',
        'rectification_core',
        'rectification_arm',
        'rectification_leg',
        'horizontal_thrust_main',
        'vertical_thrust',
        'horizontal_thrust_side',
        'horizontal_thrust_back',
        'quick_boost_main',
        'quick_boost_side',
        'quick_boost_back',
        'quick_boost_overed',
        'turning_ability',
        'stability_head',
        'stability_core',
        'stability_legs',
    ]

    tuning_values = {}
    for i, label in enumerate(tuning_labels):
        value = block[LOCAL_TUNING_OFFSET + i]
        tuning_values[label] = value  # Value should be in range 0-50

    return tuning_values


def extract_thumbnail(schematic_block: bytes) -> bytes:
    """
    Extracts the thumbnail data from a given schematic block.

    The schematic block is a chunk of data representing one schematic
    from the DESDOC.DAT file. This function isolates and returns the
    raw thumbnail data from within that block.

    Args:
        schematic_block: A bytes object representing a single schematic block.

    Returns:
        A bytes object containing the full thumbnail data (header and image).
    """
    # The absolute start of the first schematic is 0x148.
    # The absolute start of the first thumbnail is 0x200C.
    # The local offset is the difference: 0x200C - 0x148 = 0x1EC4.
    THUMBNAIL_LOCAL_OFFSET = 0x1EC4

    # The total size of the thumbnail data is 0x4010 bytes
    # (0x10 header + 0x4000 image data).
    THUMBNAIL_SIZE = 0x4010

    # Slice the block to get the thumbnail data
    thumbnail_data = schematic_block[THUMBNAIL_LOCAL_OFFSET:
                                     THUMBNAIL_LOCAL_OFFSET + THUMBNAIL_SIZE]

    return thumbnail_data


def bytes_to_image(thumbnail_bytes: bytes) -> Image.Image:
    """
    Converts raw ACFA thumbnail bytes into a Pillow Image object.

    Args:
        thumbnail_bytes: The 16400 bytes (0x4010) of thumbnail data.

    Returns:
        A Pillow Image object.
    """
    if len(thumbnail_bytes) != 0x4010:
        raise ValueError("Thumbnail data must be 0x4010 bytes long.")

    # The actual image data starts after the 16-byte header.
    dxt1_data = thumbnail_bytes[0x10:]

    # We need to construct a valid DDS header to make this readable by Pillow.
    # DDS header is 128 bytes.
    dds_header = bytearray(128)
    struct.pack_into('<4sI', dds_header, 0, b'DDS ', 124)  # Magic, Size
    # Flags
    struct.pack_into('<I', dds_header, 8, 0x1 | 0x2 | 0x4 | 0x1000 | 0x80000)
    struct.pack_into('<I', dds_header, 12, 128)  # Height
    struct.pack_into('<I', dds_header, 16, 256)  # Width
    struct.pack_into('<I', dds_header, 20, 16384)  # LinearSize
    # PixelFormat Sub-structure
    struct.pack_into('<I', dds_header, 76, 32)  # PixelFormat Size
    struct.pack_into('<I', dds_header, 80, 0x4)  # PixelFormat Flags (FourCC)
    struct.pack_into('<4s', dds_header, 84, b'DXT1')  # FourCC
    # Caps
    struct.pack_into('<I', dds_header, 108, 0x1000)  # DDSCAPS_TEXTURE

    dds_data = bytes(dds_header) + dxt1_data

    # Use an in-memory buffer to read the DDS data
    buffer = io.BytesIO(dds_data)
    image = Image.open(buffer)
    return image


def image_to_bytes(image: Image.Image) -> bytes:
    """
    Converts a Pillow Image object back into raw ACFA thumbnail bytes
    using the Wand/ImageMagick library for reliable compression.
    """
    if image.size != (256, 128):
        raise ValueError("Image must be 256x128 pixels.")

    # Convert the Pillow Image to a raw byte buffer (RGBA)
    img_rgba_bytes = image.convert('RGBA').tobytes()

    # Use a temporary file path for the DDS conversion
    with tempfile.NamedTemporaryFile(suffix='.dds', delete=False) as tmp_file:
        tmp_dds_path = tmp_file.name

    try:
        # Create a new Wand image and explicitly import the raw pixels.
        with WandImage(width=256, height=128) as img_wand:
            img_wand.import_pixels(data=img_rgba_bytes, channel_map='RGBA')

            # CRITICAL: Explicitly disable mipmap generation for the DDS file.
            img_wand.options['dds:mipmaps'] = '0'

            # Set the compression algorithm to DXT1 (BC1) and format to DDS
            img_wand.compression = 'dxt1'
            img_wand.format = 'dds'
            img_wand.save(filename=tmp_dds_path)

        # Read the temporary DDS file back into memory
        with open(tmp_dds_path, 'rb') as f:
            dds_data = f.read()

    finally:
        # Clean up the temporary file
        os.remove(tmp_dds_path)

    # Strip the 128-byte DDS header to get the raw DXT1 data
    DDS_HEADER_SIZE = 128
    dxt1_data = dds_data[DDS_HEADER_SIZE:]

    # Verify the size of the compressed data
    expected_size = 16384
    if len(dxt1_data) != expected_size:
        raise RuntimeError(
            f"DXT1 compression produced unexpected size: {len(dxt1_data)} bytes. Expected {expected_size}.")

    # Prepend the constant 16-byte ACFA header
    return ACFA_THUMBNAIL_HEADER + dxt1_data


def replace_thumbnail(schematic_block: bytes, new_thumbnail_bytes: bytes) -> bytes:
    """
    Replaces the thumbnail data within a schematic block.

    Args:
        schematic_block: The original schematic block bytes.
        new_thumbnail_bytes: The new thumbnail data (must be 0x4010 bytes).

    Returns:
        A new schematic block bytes object with the thumbnail replaced.
    """
    if len(new_thumbnail_bytes) != 0x4010:
        raise ValueError("New thumbnail data must be 0x4010 bytes long.")

    THUMBNAIL_LOCAL_OFFSET = 0x1EC4
    THUMBNAIL_SIZE = 0x4010

    # Create a mutable copy and replace the thumbnail data
    mutable_block = bytearray(schematic_block)
    mutable_block[THUMBNAIL_LOCAL_OFFSET:THUMBNAIL_LOCAL_OFFSET +
                  THUMBNAIL_SIZE] = new_thumbnail_bytes

    return bytes(mutable_block)


def save_schematic_block_as_ac4a(hex_block: bytes):
    """
    Saves a single schematic block to  'output/{schematic_name}_{designer_name}.ac4a' file.
    """
    sch_data = display_schematic_info(hex_block)
    schematic_name = sch_data['name']
    designer_name = sch_data['designer']
    output_path = f"output/{schematic_name}_{designer_name}.ac4a"

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "wb") as f:
        f.write(hex_block)


def load_schematic_block_from_ac4a(file_path: str) -> bytes:
    """
    Loads a schematic block from a .ac4a file.
    Returns the raw bytes representing the schematic block.
    """
    with open(file_path, "rb") as f:
        return f.read()
    

def insert_schematic(ac4a_path, desdoc_path):
    BLOCK_SIZE = 24280
    SCHEMATIC_COUNT_OFFSET = 5
    FIRST_MARKER_OFFSET = 0x148

    # Load files
    ac4a_data = load_file(ac4a_path)
    desdoc_data = bytearray(load_file(desdoc_path))

    # Extract current schematic count
    current_count = desdoc_data[SCHEMATIC_COUNT_OFFSET]
    print(f"Current active schematic count: {current_count}")

    # Calculate insertion offset
    insertion_offset = FIRST_MARKER_OFFSET + (current_count * BLOCK_SIZE)

    # Validate insertion space
    if insertion_offset + BLOCK_SIZE > len(desdoc_data):
        print("Error: Not enough space to insert schematic.")
        return

    # Insert schematic data
    desdoc_data[insertion_offset:insertion_offset +
                BLOCK_SIZE] = ac4a_data[:BLOCK_SIZE]

    # Update schematic count
    desdoc_data[SCHEMATIC_COUNT_OFFSET] += 1
    print(
        f"Inserted at slot {current_count + 1} (offset {hex(insertion_offset)}). New count: {desdoc_data[SCHEMATIC_COUNT_OFFSET]}")

    # Save modified DESDOC.DAT
    save_file(desdoc_path, desdoc_data)


def hex_dump(data: bytes, width: int = 16) -> str:
    lines = []
    for offset in range(0, len(data), width):
        chunk = data[offset:offset+width]
        hex_part = ' '.join(f'{b:02X}' for b in chunk)
        ascii_part = ''.join(chr(b) if 32 <= b < 127 else '.' for b in chunk)
        lines.append(f'{offset:08X}  {hex_part:<{width*3}}  {ascii_part}')
    return '\n'.join(lines)

dict_keys(['Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster', 'Back Booster', 'Side Booster', 'Overed Booster', 'Arm Unit', 'Back Unit', 'Shoulder Unit', 'Stabilizer Head Top', 'Stabilizer Head Side', 'Stabilizer Core Upper', 'Stabilizer Core Lower', 'Stabilizer Arms', 'Stabilizer Legs Back', 'Stabilizer Legs Upper', 'Stabilizer Legs Middle', 'Stabilizer Legs Lower'])


In [33]:
loaded_block = load_schematic_block_from_ac4a(
    "output/Sbeu Tarakan_Vlabus.ac4a")

# Verify loaded block content by displaying its information
display_schematic_info(loaded_block)

{'name': 'Sbeu Tarakan',
 'designer': 'Vlabus',
 'category': 2,
 'timestamp': 63883041354520147,
 'parts': [{'category': 'Head', 'part_id': '5010', 'part_name': 'SOLUH-HEAD'},
  {'category': 'Core', 'part_id': '1031', 'part_name': 'CR-LAHIRE'},
  {'category': 'Arms', 'part_id': '0061', 'part_name': 'ARGYROS/XA'},
  {'category': 'Legs', 'part_id': '6011', 'part_name': 'XLG-SOBRERO'},
  {'category': 'FCS', 'part_id': '0020', 'part_name': 'YELLOWSTONE03'},
  {'category': 'Generator', 'part_id': '2030', 'part_name': 'LINSTANT/G'},
  {'category': 'Main Booster',
   'part_id': '0031',
   'part_name': 'GAN02-NSS-M.CG'},
  {'category': 'Back Booster', 'part_id': '2010', 'part_name': '03-AALIYAH/B'},
  {'category': 'Side Booster', 'part_id': '1041', 'part_name': 'AB-LAHIRE'},
  {'category': 'Overed Booster', 'part_id': '0041', 'part_name': 'ARGYROS/AO'},
  {'category': 'Right Arm Unit', 'part_id': '5091', 'part_name': 'LARE'},
  {'category': 'Left Arm Unit', 'part_id': '2111', 'part_name': '067

In [4]:
file_path = "sch_data/tab_sch_data/DESDOC.DAT"
blocks = extract_active_schematic_blocks(file_path)
print(f"Detected {len(blocks)} active schematic(s).")

for idx, block in enumerate(blocks, 1):
    print(f"Slot {idx}:")
    print(display_schematic_info(block))

Detected 10 active schematic(s).
Slot 1:
{'name': 'U1TEST', 'designer': 'Unknown', 'category': 1, 'timestamp': 63882922127095465, 'parts': [{'category': 'Head', 'part_id': '1041', 'part_name': 'HD-LANCEL'}, {'category': 'Core', 'part_id': '1021', 'part_name': 'CR-LANCEL'}, {'category': 'Arms', 'part_id': '1031', 'part_name': 'AM-LANCEL'}, {'category': 'Legs', 'part_id': '1031', 'part_name': 'LG-LANCEL'}, {'category': 'FCS', 'part_id': '1010', 'part_name': 'FS-HOGIRE'}, {'category': 'Generator', 'part_id': '1010', 'part_name': 'GN-HOGIRE'}, {'category': 'Main Booster', 'part_id': '1020', 'part_name': 'CB-HOGIRE'}, {'category': 'Back Booster', 'part_id': '1010', 'part_name': 'LB-HOGIRE'}, {'category': 'Side Booster', 'part_id': '1030', 'part_name': 'AB-HOGIRE'}, {'category': 'Overed Booster', 'part_id': '1031', 'part_name': 'KRB-PALLAS'}, {'category': 'Right Arm Unit', 'part_id': '5071', 'part_name': 'LABIATA'}, {'category': 'Left Arm Unit', 'part_id': '1010', 'part_name': 'RF-R100'}, {'

In [58]:
save_schematic_block_as_ac4a(blocks[6])

In [75]:
insert_schematic("output/Sbeu Tarakan_Vlabus.ac4a",
                 "sch_data/tab_sch_data/DESDOC.DAT")

Current active schematic count: 9
Inserted at slot 10 (offset 0x356e0). New count: 10


In [3]:
def swap_part_in_ac4a_file(file_path, part_category, new_part_id):
    """
    Reads an .ac4a file, swaps a part, and saves it back to the same file.

    :param file_path: Path to the .ac4a schematic file.
    :param part_category: The category of the part to swap (e.g., 'Head', 'Core').
    :param new_part_id: The new part ID (as a string or int).
    """
    
    def _swap_part_in_block(block, part_category, new_part_id):
        # This is a helper function nested inside for self-containment.
        LOCAL_PARTS_OFFSET = 0xD8
        PART_ENTRY_SIZE = 2
        display_labels = [
            'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
            'Back Booster', 'Side Booster', 'Overed Booster',
            'Right Arm Unit', 'Left Arm Unit', 'Right Back Unit',
            'Left Back Unit', 'Shoulder Unit'
        ]

        try:
            part_index = display_labels.index(part_category)
        except ValueError:
            raise ValueError(f"Invalid part category: {part_category}. Must be one of {display_labels}")

        offset = LOCAL_PARTS_OFFSET + part_index * PART_ENTRY_SIZE
        
        try:
            part_id_num = int(new_part_id)
        except ValueError:
            raise ValueError(f"Invalid part ID: {new_part_id}. Must be a number.")

        new_part_bytes = part_id_num.to_bytes(2, byteorder='big')

        mutable_block = bytearray(block)
        mutable_block[offset:offset + PART_ENTRY_SIZE] = new_part_bytes
        return bytes(mutable_block)

    # Main logic for the file-based swap
    print(f"--- Modifying {file_path} ---")
    
    # 1. Load the schematic block from the file
    try:
        original_block = load_schematic_block_from_ac4a(file_path)
        print("Original file loaded.")
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return

    # 2. Perform the swap in the loaded block
    modified_block = _swap_part_in_block(original_block, part_category, new_part_id)
    print(f"Swapped '{part_category}' to part ID {new_part_id}.")

    # 3. Save the modified block back to the original file
    save_file(file_path, modified_block)
    print(f"Successfully saved changes to {file_path}.")
    print("")


# --- Example Usage ---

# Specify the file to modify
ac4a_file_to_edit = "output/AfgankaXD_RAPTOR23512.ac4a"

# Check if the file exists before trying to modify it
if os.path.exists(ac4a_file_to_edit):
    # 1. Show the parts list *before* the change
    print("--- BEFORE ---")
    before_block = load_schematic_block_from_ac4a(ac4a_file_to_edit)
    before_info = display_schematic_info(before_block)
    # A more compact print for parts
    for part in before_info['parts']:
        print(f"{part['category']}: {part['part_id']} ({part['part_name']})")
    print("")

    # 2. Call the function to swap the 'Head' to part ID '1010'
    swap_part_in_ac4a_file(ac4a_file_to_edit, 'Right Back Unit', '2030')

    # 3. Show the parts list *after* the change to verify
    print("--- AFTER ---")
    after_block = load_schematic_block_from_ac4a(ac4a_file_to_edit)
    after_info = display_schematic_info(after_block)
    for part in after_info['parts']:
        print(f"{part['category']}: {part['part_id']} ({part['part_name']})")
        
    # Example of swapping it back
    # print("--- Swapping back for demonstration ---")
    # swap_part_in_ac4a_file(ac4a_file_to_edit, 'Head', '3010') # Assuming 3010 was the original

else:
    print(f"Example file not found: {ac4a_file_to_edit}")
    print("Please ensure the file exists, for example by running the cell that calls 'save_schematic_block_as_ac4a'.")

--- BEFORE ---
Head: 3010 (047AN02)
Core: 5021 (EKHAZAR-CORE)
Arms: 7011 (WHITE-GLINT/ARMS)
Legs: 0061 (ARGYROS/L)
FCS: 1020 (FS-JUDITH)
Generator: 1051 (GN-SOBRERO)
Main Booster: 2010 (S04-VIRTUE)
Back Booster: 4021 (BB11-LATONA)
Side Booster: 4010 (SB128-SCHEDAR)
Overed Booster: 1010 (KB-JUDITH)
Right Arm Unit: 0030 (GAN01-SS-WBP)
Left Arm Unit: 0101 (GAN01-SS-WGP)
Right Back Unit: 2030 (INSOLENCE)
Left Back Unit: 0231 (OIGAMI)
Shoulder Unit: 0000 (Unknown ID 0000)

--- Modifying output/AfgankaXD_RAPTOR23512.ac4a ---
Original file loaded.
Swapped 'Right Back Unit' to part ID 2030.
Successfully saved changes to output/AfgankaXD_RAPTOR23512.ac4a.

--- AFTER ---
Head: 3010 (047AN02)
Core: 5021 (EKHAZAR-CORE)
Arms: 7011 (WHITE-GLINT/ARMS)
Legs: 0061 (ARGYROS/L)
FCS: 1020 (FS-JUDITH)
Generator: 1051 (GN-SOBRERO)
Main Booster: 2010 (S04-VIRTUE)
Back Booster: 4021 (BB11-LATONA)
Side Booster: 4010 (SB128-SCHEDAR)
Overed Booster: 1010 (KB-JUDITH)
Right Arm Unit: 0030 (GAN01-SS-WBP)
Left Arm U

In [20]:
import random
import os


def randomize_schematic_parts(file_path, part_mapping, new_name=None):
    """
    Reads an .ac4a file, randomizes its core parts, optionally renames it, 
    and saves it back to a new file, deleting the old one.
    Excludes debug parts (IDs starting with '9').

    :param file_path: Path to the .ac4a schematic file.
    :param part_mapping: The dictionary of all parts, loaded from the text file.
    :param new_name: An optional new name for the schematic.
    """
    # This function assumes that other necessary functions like
    # `load_schematic_block_from_ac4a`, `save_file`, and `display_schematic_info`
    # are already defined in the notebook.

    LOCAL_PARTS_OFFSET = 0xD8
    PART_ENTRY_SIZE = 2
    NAME_OFFSET = 1
    NAME_SIZE = 96  # 48 wchar_t = 96 bytes in UTF-16

    lookup_keys = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Arm Unit', 'Arm Unit', 'Back Unit', 'Back Unit', 'Shoulder Unit'
    ]
    display_labels = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Right Arm Unit', 'Left Arm Unit', 'Right Back Unit',
        'Left Back Unit', 'Shoulder Unit'
    ]

    try:
        original_block = load_schematic_block_from_ac4a(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return

    mutable_block = bytearray(original_block)
    print(f"--- Randomizing {file_path} ---")

    # Get original designer name for the new filename
    original_info = display_schematic_info(original_block)
    designer_name = original_info['designer']

    # Handle renaming if a new name is provided
    if new_name:
        if len(new_name) > (NAME_SIZE // 2) - 1:
            new_name = new_name[:(NAME_SIZE // 2) - 1]
            print(f"Warning: Name truncated to '{new_name}'")

        encoded_name = new_name.encode('utf-16-le')
        name_buffer = bytearray(NAME_SIZE)
        name_buffer[:len(encoded_name)] = encoded_name
        mutable_block[NAME_OFFSET:NAME_OFFSET + NAME_SIZE] = name_buffer
        print(f"  Renamed schematic to: {new_name}")

    for i, (lookup_key, display_label) in enumerate(zip(lookup_keys, display_labels)):
        part_id_list = [
            part_id for part_id in part_mapping.get(lookup_key, {}).keys()
            if not part_id.startswith('9')
        ]
        if not part_id_list:
            print(
                f"Warning: No valid parts found for category '{lookup_key}'. Skipping.")
            continue

        random_part_id_str = random.choice(part_id_list)
        random_part_id_num = int(random_part_id_str)
        offset = LOCAL_PARTS_OFFSET + i * PART_ENTRY_SIZE
        new_part_bytes = random_part_id_num.to_bytes(2, byteorder='big')
        mutable_block[offset:offset + PART_ENTRY_SIZE] = new_part_bytes
        part_name = part_mapping[lookup_key][random_part_id_str]
        print(f"  {display_label}: {random_part_id_str} ({part_name})")

    # Determine the output path
    if new_name:
        output_dir = os.path.dirname(file_path)
        new_filename = f"{new_name}.ac4a"
        output_path = os.path.join(output_dir, new_filename)
    else:
        output_path = file_path

    save_file(output_path, bytes(mutable_block))
    print(f"\nSuccessfully saved randomized schematic to {output_path}.")

    # If a new name was provided, delete the old file

In [21]:
# --- Example Usage for Randomizer ---

# Specify the file to randomize and the new name
ac4a_file_to_randomize = "output/BBBB_Unknown.ac4a"
new_schematic_name = "Randomized AC"

# Check if the file exists
if os.path.exists(ac4a_file_to_randomize):
    # 1. Show the parts list *before* the change
    print("--- BEFORE RANDOMIZATION ---")
    before_block = load_schematic_block_from_ac4a(ac4a_file_to_randomize)
    before_info = display_schematic_info(before_block)
    # Get designer name for verification
    designer_name = before_info['designer']
    print(f"Name: {before_info['name']}")
    for part in before_info['parts']:
        print(f"  {part['category']}: {part['part_id']} ({part['part_name']})")
    print("")

    # 2. Call the randomizer function with a new name
    randomize_schematic_parts(ac4a_file_to_randomize,
                              part_mapping, new_name=new_schematic_name)
    print("")

    # 3. Construct the new file path and verify the changes
    new_file_path = os.path.join(
        "output", f"{new_schematic_name}.ac4a")

    print("--- AFTER RANDOMIZATION ---")
    if os.path.exists(new_file_path):
        after_block = load_schematic_block_from_ac4a(new_file_path)
        after_info = display_schematic_info(after_block)
        print(f"Name: {after_info['name']} (Saved to: {new_file_path})")
        for part in after_info['parts']:
            print(
                f"  {part['category']}: {part['part_id']} ({part['part_name']})")
    else:
        print(f"Error: New file not found at '{new_file_path}'")
else:
    print(f"Example file not found: {ac4a_file_to_randomize}")
    print("Please ensure the file exists, for example by running a cell that calls 'save_schematic_block_as_ac4a'.")

--- BEFORE RANDOMIZATION ---
Name: BBBB
  Head: 2010 (03-AALIYAH/H)
  Core: 2010 (03-AALIYAH/C)
  Arms: 2010 (03-AALIYAH/A)
  Legs: 2010 (03-AALIYAH/L)
  FCS: 2010 (INBLUE)
  Generator: 2010 (03-AALIYAH/G)
  Main Booster: 2020 (03-AALIYAH/M)
  Back Booster: 2010 (03-AALIYAH/B)
  Side Booster: 2020 (03-AALIYAH/S)
  Overed Booster: 2010 (03-AALIYAH/O)
  Right Arm Unit: 2020 (01-HITMAN)
  Left Arm Unit: 2040 (02-DRAGONSLAYER)
  Right Back Unit: 0000 (Unknown ID 0000)
  Left Back Unit: 2020 (TRESOR)
  Shoulder Unit: 0000 (Unknown ID 0000)

--- Randomizing output/BBBB_Unknown.ac4a ---
  Renamed schematic to: Randomized AC
  Head: 0020 (KIRITUMI-H)
  Core: 1031 (CR-LAHIRE)
  Arms: 5010 (SOLUH-ARMS)
  Legs: 4030 (HILBERT-G7L)
  FCS: 2010 (INBLUE)
  Generator: 0041 (ARGYROS/G)
  Main Booster: 0010 (GAN01-SS-ML.CG)
  Back Booster: 1021 (LB-LAHIRE)
  Side Booster: 4010 (SB128-SCHEDAR)
  Overed Booster: 0051 (I-RIGEL/AO)
  Right Arm Unit: 0070 (SAKUNAMI)
  Left Arm Unit: 0151 (ALLEGHENY01)
  Righ

In [34]:
thumbnail_data = extract_thumbnail(loaded_block)
print(f"Extracted thumbnail data size: {len(thumbnail_data)} bytes")
print(hex_dump(thumbnail_data))  # Print first 64 bytes of thumbnail data
image = bytes_to_image(thumbnail_data)
image.show()  # Display the image

Extracted thumbnail data size: 16400 bytes
00000000  10 00 00 80 00 00 00 00 00 00 00 00 00 00 40 00   ..............@.
00000010  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
00000020  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
00000030  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
00000040  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
00000050  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
00000060  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
00000070  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
00000080  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
00000090  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
000000A0  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
000000B0  FF FF FF FF 00 00 00 00 FF FF FF FF 00 00 00 00   ................
000000C0  FF FF FF FF 00 00 00 00

In [31]:
new_image = Image.open("amogus.png")
new_thumbnail_bytes = image_to_bytes(new_image)
modified_block = replace_thumbnail(loaded_block, new_thumbnail_bytes)
save_schematic_block_as_ac4a(modified_block)